In [2]:
from opensky_api import OpenSkyApi
api = OpenSkyApi()
s = api.get_states()
from row64tools import ramdb
import paramiko
import pandas as pd
from opensky_api import OpenSkyApi
import time
from datetime import datetime, timedelta
from sklearn.utils import resample

In [4]:
df = pd.read_csv(r"C:\Users\mikha\Downloads\fraudTrain.csv\fraudTrain.csv")

In [5]:
fraud = df[df['is_fraud'] == 1]
non_fraud = df[df['is_fraud'] == 0]
target_fraud = 50000
target_non_fraud = 50000
fraud_upsampled = resample(
    fraud,
    replace=True,       # sample with replacement
    n_samples=target_fraud,
    random_state=42
)

# non_fraud_upsampled = resample(
#     non_fraud,
#     replace=True,
#     n_samples=target_non_fraud,
#     random_state=42
# )
# df_balanced = pd.concat([fraud_upsampled, non_fraud_upsampled])
# df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
print(len(fraud_upsampled))

50000


In [7]:
# ramdb.save_from_df(df, "C:\\Users\\mikha\\cc_fraud.ramdb")
ramdb.save_from_df(fraud_upsampled, "C:\\Users\\mikha\\fraud_upsampled.ramdb")

In [8]:
# localfile = r"C:\Users\mikha\cc_fraud.ramdb"
localfile = r"C:\Users\mikha\fraud_upsampled.ramdb"
remote_path = "/var/www/ramdb/live/RAMDB.Row64/Temp/cc_fraud.ramdb"
# remote_path = "/var/www/ramdb/live/RAMDB.Row64/Stream/cc_fraud.ramdb"

# Ubuntu server credentials
hostname = "192.168.1.23"   # Replace with your Ubuntu server's local IP
port = 22
username = "row64"   # Ubuntu login username
password = "temp7"   # Or use SSH key auth

# Transmit the data
try:
    # Connect via SSH
    transport = paramiko.Transport((hostname, port))
    transport.connect(username=username, password=password)

    # Start SFTP session
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Upload file
    sftp.put(localfile, remote_path)
    print(f"✅ File uploaded to {remote_path} on {hostname}")

    # Close connection
    sftp.close()
    transport.close()

except Exception as e:
    print(f"❌ Failed to upload: {e}")

✅ File uploaded to /var/www/ramdb/live/RAMDB.Row64/Temp/cc_fraud.ramdb on 192.168.1.23
